In [2]:
print(1)

1


In [1]:
import mediapipe as mp
import numpy as np
import time
import cv2
import torch

In [4]:
# 모션 랜드마크를 리스트로 반환
def convert_landmark_to_tensor(landmarks, n_point):
    # 영상에 모션이 잡힐 경우 
    if landmarks: 
        motion_location = []
        for lm in landmarks.landmark:
            motion_location.append([lm.x, lm.y, lm.z])
        
        return motion_location
        
    # 영상에 모션이 잡히지 않을 경우
    else:
        return [[0] * 3 for _ in range(n_point)]
    

In [5]:
# sign to tensor ipynb에서 가져온 파일
keypoints = []
left_hands = []
mp_holistic = mp.solutions.holistic
video_hol = mp.solutions.holistic.Holistic(static_image_mode=False,upper_body_only=False,smooth_landmarks=True,
                                           min_detection_confidence=0.5,min_tracking_confidence=0.5)
# Prepare DrawingSpec for drawing the face landmarks later.
mp_drawing = mp.solutions.drawing_utils 
drawing_spec = mp_drawing.DrawingSpec(thickness=1, circle_radius=1)

# For webcam input:
holistic = mp_holistic.Holistic(
    min_detection_confidence=0.5, min_tracking_confidence=0.5)
# cap = cv2.VideoCapture(0)
cap = cv2.VideoCapture("videos\KETI_SL_0000000001.avi")
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)

# cap.set(cv2.CAP_PROP_FPS, 1)

total_frames = cap.get(7)
print(total_frames)

print(cap.get(cv2.CAP_PROP_FPS))

frame_rate = 10
prev = 0
i = 0
left_hand_list = []
right_hand_list = []
face_list = []
pose_list = []

start = time.time()
while cap.isOpened():
#     time_elapsed = time.time() - prev
    success, image = cap.read()

#     if time_elapsed > 1./frame_rate:
#         prev = time.time()
    # print(type(image))
    if not success:
        print("Ignoring empty camera frame.")
    # If loading a video, use 'break' instead of 'continue'.
        break

    # Flip the image horizontally for a later selfie-view display, and convert
    # the BGR image to RGB.
    image = cv2.cvtColor(cv2.flip(image, 1), cv2.COLOR_BGR2RGB)
    # To improve performance, optionally mark the image as not writeable to
    # pass by reference.
    image.flags.writeable = False
    results = holistic.process(image)
    # Draw landmark annotation on the image.
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    mp_drawing.draw_landmarks(
      image, results.face_landmarks, mp_holistic.FACE_CONNECTIONS)
    mp_drawing.draw_landmarks(
      image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
    mp_drawing.draw_landmarks(
      image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
    mp_drawing.draw_landmarks(
      image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS)

    
    # 각 랜드마크를 리스트로 변환 후 리스트에 저장
    left_hand_list.append(convert_landmark_to_tensor(results.left_hand_landmarks, 21))
    right_hand_list.append(convert_landmark_to_tensor(results.right_hand_landmarks, 21))
    face_list.append(convert_landmark_to_tensor(results.face_landmarks, 468))
    pose_list.append(convert_landmark_to_tensor(results.pose_landmarks, 33))
    

        
        
    cv2.imshow('MediaPipe Holistic', image)
    if cv2.waitKey(5) & 0xFF == ord('q'):
        break
      
    if cv2.waitKey(5) & 0xFF == ord('w'):
        left_hand_list = []
        right_hand_list = []
        face_list = []
        pose_list = []
        break
holistic.close()
cap.release()
cv2.destroyAllWindows()
print("time :", time.time() - start)  # 현재시각 - 시작시간 = 실행 시간

141.0
29.970029
Ignoring empty camera frame.
time : 22.7850124835968


In [6]:
print(type(results.face_landmarks.landmark),'\n\n')
print(results.face_landmarks.landmark)

<class 'google.protobuf.pyext._message.RepeatedCompositeContainer'> 


[x: 0.4933699667453766
y: 0.23986779153347015
z: -0.01576869562268257
, x: 0.49248799681663513
y: 0.21629568934440613
z: -0.026771213859319687
, x: 0.49270349740982056
y: 0.2241050899028778
z: -0.014281747862696648
, x: 0.487440288066864
y: 0.1901928186416626
z: -0.01929052546620369
, x: 0.4922155737876892
y: 0.2079867124557495
z: -0.028271174058318138
, x: 0.4918307662010193
y: 0.19728538393974304
z: -0.025928502902388573
, x: 0.49094027280807495
y: 0.17172668874263763
z: -0.011480715125799179
, x: 0.4550502598285675
y: 0.16976913809776306
z: 0.0077428692020475864
, x: 0.4905138909816742
y: 0.15251153707504272
z: -0.007585086859762669
, x: 0.4902762174606323
y: 0.14166474342346191
z: -0.008274619467556477
, x: 0.48873206973075867
y: 0.09866185486316681
z: -0.0012805070728063583
, x: 0.4935084879398346
y: 0.2436017096042633
z: -0.015300055034458637
, x: 0.49359622597694397
y: 0.24655739963054657
z: -0.01367429178208

In [7]:
def pipe(t):
    t= t[:120] # seq_size가 변동되는걸 처리 못해서 임시로 고정해둠
    # batch seq value
    return t.view(1,-1,t.shape[1]*t.shape[2])
    

In [8]:
face_tensor = torch.FloatTensor(face_list)
print(face_tensor.shape)
print(face_tensor)

torch.Size([141, 468, 3])
tensor([[[ 0.4942,  0.2401, -0.0146],
         [ 0.4936,  0.2165, -0.0268],
         [ 0.4938,  0.2240, -0.0140],
         ...,
         [ 0.5016,  0.1703, -0.0024],
         [ 0.5297,  0.1595,  0.0079],
         [ 0.5321,  0.1573,  0.0080]],

        [[ 0.4948,  0.2390, -0.0156],
         [ 0.4943,  0.2153, -0.0265],
         [ 0.4943,  0.2231, -0.0141],
         ...,
         [ 0.5015,  0.1707, -0.0015],
         [ 0.5289,  0.1604,  0.0091],
         [ 0.5313,  0.1579,  0.0093]],

        [[ 0.4948,  0.2386, -0.0158],
         [ 0.4942,  0.2146, -0.0268],
         [ 0.4943,  0.2225, -0.0144],
         ...,
         [ 0.5013,  0.1706, -0.0015],
         [ 0.5287,  0.1604,  0.0090],
         [ 0.5312,  0.1580,  0.0092]],

        ...,

        [[ 0.4934,  0.2399, -0.0158],
         [ 0.4925,  0.2162, -0.0267],
         [ 0.4927,  0.2241, -0.0143],
         ...,
         [ 0.4994,  0.1721, -0.0015],
         [ 0.5268,  0.1611,  0.0089],
         [ 0.5292,  0.15

In [9]:
lt = pipe(torch.FloatTensor(left_hand_list))
rt = pipe(torch.FloatTensor(right_hand_list))
ft = pipe(torch.FloatTensor(face_list))
pt = pipe(torch.FloatTensor(pose_list))


In [10]:
print(lt.shape)

torch.Size([1, 120, 63])


In [11]:
def load_tensor(numRange):

    lt_list = []
    rt_list = []
    ft_list = []
    pt_list = []
    tmp_base = "0000000000"
    for x in range(1,numRange+1):
        base = tmp_base[:-1*(len(str(x)))]

        lt_list.append(torch.load("./output/tensor/KETI_SL_"+base+str(x)+"/left_hand.pt"))
        rt_list.append(torch.load("./output/tensor/KETI_SL_"+base+str(x)+"/right_hand.pt"))
        ft_list.append(torch.load("./output/tensor/KETI_SL_"+base+str(x)+"/face.pt"))
        pt_list.append(torch.load("./output/tensor/KETI_SL_"+base+str(x)+"/pose.pt"))
    return lt_list,rt_list,ft_list,pt_list


In [13]:
lt_list,rt_list,ft_list,pt_list = load_tensor(20)

In [14]:
print(lt_list[0].shape)

torch.Size([141, 21, 3])


In [12]:

print(lt.shape, rt.shape, ft.shape, pt.shape)

torch.Size([1, 120, 63]) torch.Size([1, 120, 63]) torch.Size([1, 120, 1404]) torch.Size([1, 120, 99])


In [15]:

lt_list = list(map(pipe,lt_list))
rt_list = list(map(pipe,rt_list))
ft_list = list(map(pipe,ft_list))
pt_list = list(map(pipe,pt_list))

In [16]:
print(lt_list[0].shape)

torch.Size([1, 120, 63])


In [2]:
print(1)

1


In [17]:
class Net(torch.nn.Module):
    '''
    현재 모델 상황
    
    구조
    left, right, face, pose 전부 각각을 rnn -> dnn 순으로 입력후
    모두 concat하고 flatten 한뒤 dnn에 입력함

    rnn -> relu -> dnn -> relu -> concat -> flat -> dnn -> softmax 순

    구성 이유
    왼손, 오른손, 얼굴, 포즈를 각각 rnn에 돌려서 특정 패턴으로 인식하고 
    해당 패턴을 조합하여 값을 내놓는다 라는 가정하에 만들었음

    제한 사항

    input shape를 고정하고
    output 또한 one hot vector로 출력되며 해당값은 1~20사이 값만 출력됨

    수정이 필요한 사항
    1.  seq사이즈를 고정하지 않고 하려 했으내 nn.Embeding은 long타입이 필요하며
        다른 방식으로 seq사이즈를 고정하는 방식을 찾이 못했음

    2.  output이 지금은 softmax를 활용한 단순 분류임 text로 변환하려는 작업이 추가적으로 필요함

    '''
    def __init__(self,seq, hidden_size, output_size):
        super(Net, self).__init__()
        # (batch_size,seq, 21*3)
        # (seq, batch, feature)
        # (batch_size,seq, 21*3)
        # (batch_size,seq, 468*3)   
        # (batch_size,seq, 33*3)

        # 임배딩이 필요함 이때 GCN사용


        self.lt = torch.nn.GRU(21*3, hidden_size,batch_first=True) # GRU 셀 구현
        self.rt = torch.nn.GRU(21*3, hidden_size,batch_first=True) # GRU 셀 구현
        self.ft = torch.nn.GRU(468*3, hidden_size,batch_first=True) # GRU 셀 구현
        self.pt = torch.nn.GRU(33*3, hidden_size,batch_first=True) # GRU 셀 구현 
        
        self.trans_encode =torch.nn.TransformerEncoder()
        self trans_decode = torch.nn.TransformerDecoder()
       

        self.relu = torch.nn.ReLU()
        self.soft = torch.nn.Softmax(dim=1)
        self.dnnlt = torch.nn.Linear(hidden_size, output_size, bias=True) # 출력층 구현
        self.dnnrt = torch.nn.Linear(hidden_size, output_size, bias=True) # 출력층 구현
        self.dnnft = torch.nn.Linear(hidden_size, output_size, bias=True) # 출력층 구현
        self.dnnpt = torch.nn.Linear(hidden_size, output_size, bias=True) # 출력층 구현

        # (1,120,20) 2400
        # (1, 20)
        # (1, 80)

        self.flat   = torch.nn.Flatten()
        self.outDnn = torch.nn.Linear(seq * output_size*4,20,bias=True)

        
    def forward(self,lt,rt,ft,pt ): # 구현한 RNN 셀과 출력층을 연결
        rlt, _status =self.lt(lt)
        rrt, _status =self.rt(rt)
        rft, _status =self.ft(ft)
        rpt, _status =self.pt(pt)

        rlt=self.relu(rlt)
        rrt=self.relu(rrt)
        rft=self.relu(rft)
        rpt=self.relu(rpt)


        dlt= self.dnnlt(rlt)
        drt= self.dnnrt(rrt)
        dft= self.dnnft(rft)
        dpt= self.dnnpt(rpt)

        
        dlt=self.relu(dlt)
        drt=self.relu(drt)
        dft=self.relu(dft)
        dpt=self.relu(dpt)

        fullDnn = torch.cat((dlt, drt, dft, dpt),dim=2)
        out = self.flat(fullDnn)
        out = self.outDnn(out)
        out = self.soft(out)

        return out

In [18]:
net = Net(lt_list[0].shape[1], 60,200)
net.forward

<bound method Net.forward of Net(
  (lt): RNN(63, 60, batch_first=True)
  (rt): RNN(63, 60, batch_first=True)
  (ft): RNN(1404, 60, batch_first=True)
  (pt): RNN(99, 60, batch_first=True)
  (relu): ReLU()
  (soft): Softmax(dim=1)
  (dnnlt): Linear(in_features=60, out_features=200, bias=True)
  (dnnrt): Linear(in_features=60, out_features=200, bias=True)
  (dnnft): Linear(in_features=60, out_features=200, bias=True)
  (dnnpt): Linear(in_features=60, out_features=200, bias=True)
  (flat): Flatten(start_dim=1, end_dim=-1)
  (outDnn): Linear(in_features=96000, out_features=20, bias=True)
)>

In [19]:
learning_rate = 1e-3
loss_fn = torch.nn.BCELoss  # Binary Cross Entropy 강화학습때 했던 log사용하는 loss 함수
                            # https://pytorch.org/docs/stable/generated/torch.nn.BCELoss.html
optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate) 

In [20]:
y_data = torch.FloatTensor(np.eye(20))
print(y_data[0])

tensor([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0.])


In [21]:
epochs = 100
losses = [] 
for i in range(epochs): 
    for x in range(len(lt_list)):
        predict = net(lt_list[x], rt_list[x], ft_list[x], pt_list[x])    # [1,20] ont-hot vector
        X = predict[0] # [20] 으로 변경
        Y = y_data[x]  # [20] 짜리 단위 행렬
        loss = loss_fn()(X,Y)   # Binary Cross Entropy 강화학습때 했던 log사용하는 loss 함수

        optimizer.zero_grad()
        loss.backward()
        losses.append(loss.item())
        optimizer.step()
    print("epoch : {0} , loss : {1} ".format(i,sum(losses[i*20:(i+1)*20])))

epoch : 0 , loss : 7.537634804844856 
epoch : 1 , loss : 6.3480765372514725 
epoch : 2 , loss : 6.492609590291977 
epoch : 3 , loss : 4.666203930974007 
epoch : 4 , loss : 2.771341484040022 
epoch : 5 , loss : 1.0543707867327612 
epoch : 6 , loss : 0.3832622717563936 
epoch : 7 , loss : 0.030057463719913358 
epoch : 8 , loss : 0.00861805072736388 
epoch : 9 , loss : 0.0064380973355468996 


KeyboardInterrupt: 

In [19]:
model = Net(lt_list[0].shape[1], 60,200)
model.load_state_dict(torch.load("./output/model/model.pt"),strict=False) # model load 하기

<All keys matched successfully>

In [20]:
net(lt_list[x], rt_list[x], ft_list[x], pt_list[x]) 

tensor([[1.0000e+00, 1.2892e-43, 8.2677e-44, 9.8091e-44, 6.4460e-44, 7.4269e-44,
         6.1657e-44, 6.5861e-44, 6.7262e-44, 7.8473e-44, 9.1084e-44, 7.8473e-44,
         6.3058e-44, 7.4269e-44, 6.7262e-44, 7.7071e-44, 6.5861e-44, 7.8473e-44,
         6.5861e-44, 7.9874e-44]], grad_fn=<SoftmaxBackward>)

In [24]:
model(lt_list[x], rt_list[x], ft_list[x], pt_list[x]) , x

(tensor([[3.8699e-09, 2.1393e-09, 2.3582e-08, 5.8050e-09, 4.8973e-08, 2.9401e-08,
          1.3573e-09, 4.4913e-08, 1.9066e-09, 1.0000e+00, 2.9191e-09, 1.3987e-09,
          1.8078e-09, 7.1477e-09, 2.0885e-09, 9.3883e-10, 2.9283e-09, 2.5023e-10,
          1.0199e-09, 8.7668e-09]], grad_fn=<SoftmaxBackward>), 10)

In [33]:
model(lt, rt, ft, pt) , x+1

(tensor([[7.0290e-01, 1.4449e-05, 4.4292e-05, 1.2296e-03, 2.3749e-01, 2.0924e-05,
          9.1894e-06, 7.9870e-06, 2.5336e-05, 9.0888e-07, 3.5219e-07, 1.0738e-04,
          1.3813e-02, 4.2674e-05, 3.7969e-05, 2.4304e-05, 4.8775e-05, 4.3588e-02,
          3.3810e-04, 2.5872e-04]], grad_fn=<SoftmaxBackward>), 10)

In [48]:
np.argmax(model(lt,rt,ft,pt).detach())

tensor(8)

매 frame마다 tensor를 만들고

실제 모델 실행시 90frame을 넣는다라고 가정을하면

tensor를 만드는거는 값을 직접입력하는 방식으로 진행하고 

만든 tensor를 리스트에 append한다. 

그 이후 b = torch.Tensor(90,size,3)

torch.cat(tensorlist[-90:],out = b)
이런식으로 뽑는다



